In [2]:
!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org dash

  Created wheel for dash: filename=dash-1.17.0-py3-none-any.whl size=83730 sha256=44e117ee3952830b35088661a1765b559e6f848c773fb1e7dcb1c10e86f61b42
  Stored in directory: c:\users\80575\appdata\local\pip\cache\wheels\fc\16\5c\549c935a88b11f2340725457d3135bc1d0900757fec1b98031
  Created wheel for flask-compress: filename=Flask_Compress-1.8.0-py3-none-any.whl size=7221 sha256=be1189d74604e2b0099278345b86d77c210c2b12fbac1f73fa6e477043071489
  Stored in directory: c:\users\80575\appdata\local\pip\cache\wheels\93\bd\ef\e80d536d859386e52785128d4257565af4a0643b1d98376e15
  Created wheel for dash-renderer: filename=dash_renderer-1.8.3-py3-none-any.whl size=1013946 sha256=5e60eb88b7bd5a97debc84b394e1a37b791429c00750684a429eeb68cdc4ad3b
  Stored in directory: c:\users\80575\appdata\local\pip\cache\wheels\82\4d\8c\0764de014b25313cf54199f212bad4fb4b228ecfaef58be3b9
  Created wheel for dash-core-components: filename=dash_core_components-1.13.0-py3-none-any.whl size=3513146 sha256=7b77dadaf4355f51508

In [3]:
import pandas as pd
import numpy as np
import math
from IPython.display import Image
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, ALL, State, MATCH, ALLSMALLER
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

In [4]:
# read data
df = pd.read_csv('pokemon.csv')
# binary data
df.is_legendary = df.is_legendary.astype('str')
df.generation = df.generation.astype('str')

In [5]:
# data preparation : scaling score columns
scaler = MinMaxScaler()
scaler.fit(df.loc[:,['base_total','attack','sp_attack','defense','sp_defense','speed','hp']])
df.loc[:,['base_total','attack','sp_attack','defense','sp_defense','speed','hp']] = scaler.transform(df.loc[:,['base_total','attack','sp_attack','defense','sp_defense','speed','hp']])*100

In [6]:
# data preparation : poke type master
df['type_cnt'] = 2
df.loc[df.type2==np.nan,'type_cnt'] = 1
df_info = df.loc[:,['pokedex_number', 'name', 'is_legendary', 'generation', 'capture_rate', 'experience_growth','type_cnt']]
df_abil = df.loc[:,['pokedex_number', 'base_total','attack','sp_attack','defense','sp_defense','speed','hp', 'height_m', 'weight_kg']]
df_type1 = df.loc[:,['pokedex_number', 'type1']]
df_type1.columns = ['pokedex_number', 'type']
df_type2 = df.loc[df.type2.isnull()==False,['pokedex_number', 'type2']]
df_type2.columns = ['pokedex_number', 'type']
df_type = pd.concat([df_type1, df_type2],axis=0)
del df_type1, df_type2

In [7]:
# data preparation : dropdown lists
ability_list = ['base_total','attack','sp_attack','defense','sp_defense','speed','hp']
legendary_list = list(df_info.is_legendary.unique())
type_list = list(df_type.type.unique())
genreation_list = list(df_info.generation.astype('str').unique())
# against_list = list(df.columns[1:19])

In [8]:

# set app
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

In [9]:
# set layout
app.layout = html.Div([html.Div([html.Div([dcc.Dropdown(id='crossfilter-xaxis-column',
                                                        options=[{'label': i, 'value': i} for i in ability_list],
                                                        value='defense')],
                                          style={'width': '33%', 'float': 'left', 'display': 'inline-block'}),
                                 
                                 html.Div([dcc.Dropdown(id='crossfilter-yaxis-column',
                                                        options=[{'label': i, 'value': i} for i in ability_list],
                                                        value='attack')],
                                          style={'width': '33%', 'float': 'left', 'display': 'inline-block'}),
                                 
                                 html.Div([dcc.Dropdown(id='crossfilter-zaxis-column',
                                                        options=[{'label': i, 'value': i} for i in ability_list],
                                                        value='base_total')],
                                          style={'width': '33%', 'display': 'inline-block'})]),
                       
                       
                       html.Div([html.Div([dcc.Dropdown(id='is-legendary-column',
                                                        options=[{'label': i, 'value': i} for i in legendary_list],
                                                        value=legendary_list, multi=True)],
                                          style={'width': '33%', 'float': 'left', 'display': 'inline-block'}),
                                 
                                 html.Div([dcc.Dropdown(id='type-column',
                                                        options=[{'label': i, 'value': i} for i in type_list],
                                                        value=type_list, multi=True)],
                                          style={'width': '33%', 'float': 'left', 'display': 'inline-block'}),
                                 
                                 html.Div([dcc.Dropdown(id='generation-column',
                                                        options=[{'label': i, 'value': i} for i in genreation_list],
                                                        value=genreation_list, multi=True)],
                                          style={'width': '33%', 'display': 'inline-block'})]),
                       
                       
                       
                       html.Div([html.Div([dcc.Graph(id='crossfilter-indicator-scatter')],
                                          style={'width': '50%', 'float': 'left', 'display': 'inline-block'}),
                                 html.Div([dcc.Graph(id='scatterpolar')],
                                          style={'width': '50%', 'float': 'left', 'display': 'inline-block'})],
                                style={'display': 'inline-block'}),
                       
                       
                       html.Div([html.Div([html.Div([dcc.Graph(id='x-legendary')], style={'display': 'inline-block', 'height': '80%'}),
                                           html.Div([dcc.Graph(id='x-violin')], style={'display': 'inline-block', 'height': '20%'})],
                                          style={'display': 'inline-block', 'width': '33%'}),
                                 html.Div([html.Div([dcc.Graph(id='y-legendary')], style={'display': 'inline-block', 'height': '80%'}),
                                           html.Div([dcc.Graph(id='y-violin')], style={'display': 'inline-block', 'height': '20%'})],
                                          style={'display': 'inline-block', 'width': '33%'}),
                                 html.Div([html.Div([dcc.Graph(id='z-legendary')], style={'display': 'inline-block', 'height': '80%'}),
                                           html.Div([dcc.Graph(id='z-violin')], style={'display': 'inline-block', 'height': '20%'})],
                                          style={'display': 'inline-block', 'width': '33%'})
                                ])
                      ])

In [10]:
# scatter_3d
@app.callback(
    Output('crossfilter-indicator-scatter', 'figure'),
    
    [Input('crossfilter-xaxis-column', 'value'),
     Input('crossfilter-yaxis-column', 'value'),
     Input('crossfilter-zaxis-column', 'value'),
     Input('is-legendary-column', 'value'),
     Input('type-column', 'value'),
     Input('generation-column', 'value')])

def update_graph(xaxis_column_name, yaxis_column_name, zaxis_column_name, is_legendary_value, type_value, generation_value):
    # selected pokedex_numbers by dropdown
    set1 = set(df_info.loc[df_info['is_legendary'].isin(is_legendary_value),'pokedex_number'])
    set2 = set(df_type.loc[df_type['type'].isin(type_value),'pokedex_number'])
    set3 = set(df_info.loc[df_info['generation'].isin(generation_value),'pokedex_number'])
    no_list = list(set1&set2&set3)
    
    # dataframe with selected pokedex_numbers
    dff = df.loc[df['pokedex_number'].isin(no_list),:]
    # save the chart on fig
    fig = px.scatter_3d(x=dff.loc[:,xaxis_column_name],
                        y=dff.loc[:,yaxis_column_name],
                        z=dff.loc[:,zaxis_column_name],
                        hover_name=dff.name, color=dff.is_legendary, size=dff.base_total,
                       labels = {'x':xaxis_column_name, 'y':yaxis_column_name, 'z':zaxis_column_name})
    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')
    return fig

In [11]:
#  Scatterpolar
@app.callback(Output('scatterpolar', 'figure'),
    [Input('is-legendary-column', 'value'),
     Input('type-column', 'value'),
     Input('generation-column', 'value')])

def update_graph2(is_legendary_value, type_value, generation_value):
    # selected pokedex_numbers by dropdown
    set1 = set(df_info.loc[df_info['is_legendary'].isin(is_legendary_value),'pokedex_number'])
    set2 = set(df_type.loc[df_type['type'].isin(type_value),'pokedex_number'])
    set3 = set(df_info.loc[df_info['generation'].isin(generation_value),'pokedex_number'])
    no_list = list(set1&set2&set3)
    # dataframe with selected pokedex_numbers
    dff = df.loc[df['pokedex_number'].isin(no_list),:]
    norm_stat = dff.loc[dff.is_legendary=='0', ['attack','sp_attack','defense','sp_defense','speed','hp']].mean()
    legd_stat = dff.loc[dff.is_legendary=='1', ['attack','sp_attack','defense','sp_defense','speed','hp']].mean()
    # save the chart on fig
    fig = go.Figure()
    fig.add_trace(go.Scatterpolar(r=norm_stat.values, theta=norm_stat.index, fill='toself', name='Normal'))
    fig.add_trace(go.Scatterpolar(r=legd_stat.values, theta=legd_stat.index, fill='toself', name='Legendary'))
    fig.update_layout(polar=dict(radialaxis=dict(visible=True,)), showlegend=False)
    return fig

In [12]:
# bar & violin by axes
# xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
@app.callback(
    Output('x-legendary', 'figure'),
    [Input('crossfilter-xaxis-column', 'value'),
     Input('is-legendary-column', 'value'),
     Input('type-column', 'value'),
     Input('generation-column', 'value')])

def update_x_bar(xaxis_column_name,is_legendary_value, type_value, generation_value):
    # selected pokedex_numbers by dropdown
    set1 = set(df_info.loc[df_info['is_legendary'].isin(is_legendary_value),'pokedex_number'])
    set2 = set(df_type.loc[df_type['type'].isin(type_value),'pokedex_number'])
    set3 = set(df_info.loc[df_info['generation'].isin(generation_value),'pokedex_number'])
    no_list = list(set1&set2&set3)
    # dataframe with selected pokedex_numbers
    dff = df.loc[df['pokedex_number'].isin(no_list),:]
    # save the chart on fig
    fig = px.bar(y=dff.name, x=dff.loc[:,xaxis_column_name],
            hover_name=dff.name, color=dff.is_legendary)
    fig.update_xaxes(title=xaxis_column_name)
    fig.update_layout(yaxis={'categoryorder':'total ascending'})
    return fig

@app.callback(
    Output('x-violin', 'figure'),
    [Input('crossfilter-xaxis-column', 'value'),
     Input('is-legendary-column', 'value'),
     Input('type-column', 'value'),
     Input('generation-column', 'value')])
def update_x_violin(xaxis_column_name,is_legendary_value, type_value, generation_value):
    # selected pokedex_numbers by dropdown
    set1 = set(df_info.loc[df_info['is_legendary'].isin(is_legendary_value),'pokedex_number'])
    set2 = set(df_type.loc[df_type['type'].isin(type_value),'pokedex_number'])
    set3 = set(df_info.loc[df_info['generation'].isin(generation_value),'pokedex_number'])
    no_list = list(set1&set2&set3)
    # dataframe with selected pokedex_numbers
    dff = df.loc[df['pokedex_number'].isin(no_list),:]
    # save the chart on fig
    fig = go.Figure()
    fig.add_trace(go.Violin(x=dff.loc[dff.is_legendary=='0',xaxis_column_name], y = ['']*np.sum(dff.is_legendary=='0'),
                        legendgroup='0', scalegroup='0', name='0',
                        side='negative', line_color="blue"))
    fig.add_trace(go.Violin(x=dff.loc[dff.is_legendary=='1',xaxis_column_name], y = ['']*np.sum(dff.is_legendary=='1'),
                        legendgroup='1', scalegroup='1', name='1',
                        side='positive', line_color="red"))
    fig.update_traces(orientation='h')
    fig.update_xaxes(title=xaxis_column_name)
    return fig
# yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
@app.callback(
    Output('y-legendary', 'figure'),
    [Input('crossfilter-yaxis-column', 'value'),
     Input('is-legendary-column', 'value'),
     Input('type-column', 'value'),
     Input('generation-column', 'value')])
def update_y_bar(yaxis_column_name,is_legendary_value, type_value, generation_value):
    # selected pokedex_numbers by dropdown
    set1 = set(df_info.loc[df_info['is_legendary'].isin(is_legendary_value),'pokedex_number'])
    set2 = set(df_type.loc[df_type['type'].isin(type_value),'pokedex_number'])
    set3 = set(df_info.loc[df_info['generation'].isin(generation_value),'pokedex_number'])
    no_list = list(set1&set2&set3)
    # dataframe with selected pokedex_numbers
    dff = df.loc[df['pokedex_number'].isin(no_list),:]
    # save the chart on fig
    fig = px.bar(y=dff.name, x=dff.loc[:,yaxis_column_name],
            hover_name=dff.name, color=dff.is_legendary)
    fig.update_xaxes(title=yaxis_column_name)
    fig.update_layout(yaxis={'categoryorder':'total ascending'})
    return fig
@app.callback(
    Output('y-violin', 'figure'),
    [Input('crossfilter-yaxis-column', 'value'),
     Input('is-legendary-column', 'value'),
     Input('type-column', 'value'),
     Input('generation-column', 'value')])
def update_y_violin(yaxis_column_name,is_legendary_value, type_value, generation_value):
    # selected pokedex_numbers by dropdown
    set1 = set(df_info.loc[df_info['is_legendary'].isin(is_legendary_value),'pokedex_number'])
    set2 = set(df_type.loc[df_type['type'].isin(type_value),'pokedex_number'])
    set3 = set(df_info.loc[df_info['generation'].isin(generation_value),'pokedex_number'])
    no_list = list(set1&set2&set3)
    # dataframe with selected pokedex_numbers
    dff = df.loc[df['pokedex_number'].isin(no_list),:]
    # save the chart on fig
    fig = go.Figure()
    fig.add_trace(go.Violin(x=dff.loc[dff.is_legendary=='0',yaxis_column_name], y = ['']*np.sum(dff.is_legendary=='0'),
                        legendgroup='0', scalegroup='0', name='0',
                        side='negative', line_color="blue"))
    fig.add_trace(go.Violin(x=dff.loc[dff.is_legendary=='1',yaxis_column_name], y = ['']*np.sum(dff.is_legendary=='1'),
                        legendgroup='1', scalegroup='1', name='1',
                        side='positive', line_color="red"))
    fig.update_traces(orientation='h')
    fig.update_xaxes(title=yaxis_column_name)
    return fig
# zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz
@app.callback(
    Output('z-legendary', 'figure'),
    [Input('crossfilter-zaxis-column', 'value'),
     Input('is-legendary-column', 'value'),
     Input('type-column', 'value'),
     Input('generation-column', 'value')])
def update_z_bar(zaxis_column_name,is_legendary_value, type_value, generation_value):
    # selected pokedex_numbers by dropdown
    set1 = set(df_info.loc[df_info['is_legendary'].isin(is_legendary_value),'pokedex_number'])
    set2 = set(df_type.loc[df_type['type'].isin(type_value),'pokedex_number'])
    set3 = set(df_info.loc[df_info['generation'].isin(generation_value),'pokedex_number'])
    no_list = list(set1&set2&set3)
    # dataframe with selected pokedex_numbers
    dff = df.loc[df['pokedex_number'].isin(no_list),:]
    # save the chart on fig
    fig = px.bar(y=dff.name, x=dff.loc[:,zaxis_column_name],
            hover_name=dff.name, color=dff.is_legendary)
    fig.update_xaxes(title=zaxis_column_name)
    fig.update_layout(yaxis={'categoryorder':'total ascending'})
    return fig
@app.callback(
    Output('z-violin', 'figure'),
    [Input('crossfilter-zaxis-column', 'value'),
     Input('is-legendary-column', 'value'),
     Input('type-column', 'value'),
     Input('generation-column', 'value')])
def update_z_violin(zaxis_column_name,is_legendary_value, type_value, generation_value):
    # selected pokedex_numbers by dropdown
    set1 = set(df_info.loc[df_info['is_legendary'].isin(is_legendary_value),'pokedex_number'])
    set2 = set(df_type.loc[df_type['type'].isin(type_value),'pokedex_number'])
    set3 = set(df_info.loc[df_info['generation'].isin(generation_value),'pokedex_number'])
    no_list = list(set1&set2&set3)
    # dataframe with selected pokedex_numbers
    dff = df.loc[df['pokedex_number'].isin(no_list),:]
    # save the chart on fig
    fig = go.Figure()
    fig.add_trace(go.Violin(x=dff.loc[dff.is_legendary=='0',zaxis_column_name], y = ['']*np.sum(dff.is_legendary=='0'),
                        legendgroup='0', scalegroup='0', name='0',
                        side='negative', line_color="blue"))
    fig.add_trace(go.Violin(x=dff.loc[dff.is_legendary=='1',zaxis_column_name], y = ['']*np.sum(dff.is_legendary=='1'),
                        legendgroup='1', scalegroup='1', name='1',
                        side='positive', line_color="red"))
    fig.update_traces(orientation='h')
    fig.update_xaxes(title=zaxis_column_name)
    return fig

In [ ]:

if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Nov/2020 16:31:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2020 16:31:39] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_8_3m1604475039.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2020 16:31:39] "GET /_dash-component-suites/dash_renderer/react@16.v1_8_3m1604475039.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2020 16:31:39] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_8_3m1604475039.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2020 16:31:39] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_8_3m1604475039.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2020 16:31:39] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_13_0m1604475042.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2020 16:31:39] "GET /_dash-component-suites/dash_html_components/dash_html_components.v1_1_1m1604475048.min.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2020 16

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\ProgramData\Anaconda3\lib\site-packages\dash\dash.py", line 1076, in dispatch
    response.set_d

127.0.0.1 - - [04/Nov/2020 16:42:47] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\ProgramData\Anaconda3\lib\site-packages\dash\dash.py", line 1076, in dispatch
    response.set_d

127.0.0.1 - - [04/Nov/2020 16:42:47] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\ProgramData\Anaconda3\lib\site-packages\dash\dash.py", line 1076, in dispatch
    response.set_d

127.0.0.1 - - [04/Nov/2020 16:42:47] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\ProgramData\Anaconda3\lib\site-packages\dash\dash.py", line 1076, in dispatch
    response.set_d

127.0.0.1 - - [04/Nov/2020 16:42:47] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [04/Nov/2020 16:42:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2020 16:42:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2020 16:42:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2020 16:42:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2020 16:42:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2020 16:42:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2020 16:42:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2020 16:42:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2020 16:43:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2020 16:43:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2020 16:43:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.